# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

**Описание данных**   
* airports_nearest — расстояние до ближайшего аэропорта в метрах (м)   
* balcony — число балконов
* ceiling_height — высота потолков (м)  
* cityCenters_nearest — расстояние до центра города (м)  
* days_exposition — сколько дней было размещено объявление (от публикации до снятия)  
* first_day_exposition — дата публикации  
* floor — этаж  
* floors_total — всего этажей в доме  
* is_apartment — апартаменты (булев тип)  
* kitchen_area — площадь кухни в квадратных метрах (м²)  
* last_price — цена на момент снятия с публикации  
* living_area — жилая площадь в квадратных метрах (м²)  
* locality_name — название населённого пункта  
* open_plan — свободная планировка (булев тип)  
* parks_around3000 — число парков в радиусе 3 км  
* parks_nearest — расстояние до ближайшего парка (м)  
* ponds_around3000 — число водоёмов в радиусе 3 км  
* ponds_nearest — расстояние до ближайшего водоёма (м)  
* rooms — число комнат  
* studio — квартира-студия (булев тип)  
* total_area — общая площадь квартиры в квадратных метрах (м²)  
* total_images — число фотографий квартиры в объявлении

В нашем распоряжении данные сервиса Яндекс Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — провести исследовательский анализ данных, который поможет установить параметры, влияющие на цену объектов. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.   
По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта и других объектов — эти данные автоматически получены из геосервисов. Количество парков и водоёмов также заполняется без участия пользователя. 

</font><font color='Blue'><b>Пропуски в столбцах</b></font><br>

### Откройте файл с данными и изучите общую информацию. 

**Подключение библиотек**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import copy

**Изучение общей информации о полученном датафрейме**

In [2]:
data = pd.read_csv('C://Users/rdl_b/OneDrive/Рабочий стол/python/Yandex Practicum/Загрузка проектов/Проект 2/real_estate_data.csv', sep ='\t')
print("Кол-во строк, столбцов:", data.shape)
print()
print(data.info())
data_shape = data.shape[0]
pd.set_option('display.max_columns', None)
data.head()

**Посмотрим уникальные значения в каждом столбце**

In [3]:
for column in data.columns:
    print(column)
    print(data[column].unique())
    print()

In [4]:
data.hist(figsize=(15, 30))


### Предобработка данных

**Определим в каких столбацах есть пропуски**

In [5]:
data_shape = data.shape[0]
print("Всего строк в данных:", data_shape) 
for column in data.columns:
    data_isna = data[column].isna().value_counts()
    print('Количество пропусков в', column, ': ', data_shape - data_isna[0])


Количество пропусков в ceiling_height :  9195 - **Очень большое кол-во пропущенных для восстановления данных**  
 


Количество пропусков в locality_name :  49  **можно попробовать восстановить по значениям положения от центра**  
Количество пропусков в airports_nearest :  5542  
Количество пропусков в cityCenters_nearest :  5519  
Количество пропусков в parks_around3000 :  5518  
Количество пропусков в parks_nearest :  15620  
Количество пропусков в ponds_around3000 :  5518  
Количество пропусков в ponds_nearest :  14589  
Количество пропусков в days_exposition :  3181  


In [6]:
print(data[data['floors_total'].isna()]['floor'].value_counts())
data['floors_total'].value_counts()

**Этажность возможно не стали указывать т.к. этаж на котором квартира - это последний этаж в доме.  
Предлагаю заполнить этими данными столбец 'floors_total'**

In [7]:
data['floors_total'] = data['floors_total'].fillna(data['floor'])

In [8]:
data['floors_total'].isna().value_counts()

**Попуски в living_area заполним, исходя из среднего значения соотношений жилой площади к общей. Т.е. найдем процент в среднем, который составляет жилая зона от общей зоны**

In [9]:
print(data.loc[:10,['living_area', 'total_area']])

In [10]:
living_area_mean = (data['living_area']/data['total_area']).mean()
print('Среднее значение отношения жилой к нежилой площади(%):', living_area_mean)


In [11]:
data['living_area'] = data['living_area'].fillna(0)

def fill_living_area(row):
    if row[7] == 0:
        row[7] = float((row[2] * living_area_mean))
    return row

data = data.apply(fill_living_area, axis = 1)

In [12]:
print(data['living_area'].isna().value_counts())
print(data.loc[:10,['living_area', 'total_area']]) #Проверим значения

**Попуски в is_apartment заменим на False, так как отсуствие по этому пункту говорит нам, что перед нами обычная квартира**

In [13]:
data['is_apartment'] = data['is_apartment'].fillna(False)

In [14]:
data['is_apartment'].isna().value_counts()

**Пропуски в  kitchen_area заменим аналогично пропускам в living_area**

In [15]:
kitchen_area_mean = (data['kitchen_area']/data['total_area']).mean()
print('Среднее значение отношения кухни к общей площади(%):', kitchen_area_mean)

In [16]:
data['kitchen_area'] = data['kitchen_area'].fillna(0)

def fill_kitchen_area(row):
    if row[12] == 0:
        row[12] = float((row[2] * kitchen_area_mean))
    return row

data = data.apply(fill_kitchen_area, axis = 1)

In [17]:
print(data['kitchen_area'].isna().value_counts())
print(data.loc[:10,['kitchen_area', 'total_area']]) #Проверим значения

**Пропуски в  balcony заменим на 0, т.к. их не указали ввиду отсутствия балкона**

In [18]:
data['balcony'] = data['balcony'].fillna(0)

In [19]:
print(data['balcony'].isna().value_counts())

In [20]:
miss_locality_name = data[data['locality_name'].isna()] # Проверим пропущенные значения в locality_name
miss_locality_name.head(10)



**Неполучиться восстановить данные, если в cityCenters_nearest стоит NaN. Удалим эти значения.**

In [21]:
bad_miss_locality_name = miss_locality_name[miss_locality_name['cityCenters_nearest'].isna()]
bad_miss_locality_name.head()

In [22]:
bad_index = bad_miss_locality_name.index
print(bad_index)

data.drop(bad_index, inplace=True)
data.loc[2030:2035, :]

**Проведем поиск расстояния по указанному locality_name от центра Санкт-Петербурга. Начнем с категоризации местоположения**

In [23]:
print(data['locality_name'].value_counts())
data['locality_name'].unique()

**В названиях есть дубликаты, например: "посёлок Пансионат Зелёный Бор" и "поселок Пансионат Зелёный Бор". Приведем названия к одному виду.**

In [24]:
data_copy = copy.deepcopy(data)
bad_locality_name = data[data['locality_name'].isna()]

bad_index = bad_locality_name.index
print(bad_index)

data_copy.drop(bad_index, inplace=True)

def make_short_name(row):
    #print(1, row)
    split_name = row.split()
    new_name = []
    for word in split_name:
        if word[0].isupper():
            new_name.append(word)
    new_name = ' '.join(word for word in new_name)
    #print(2, new_name)
    return new_name

    
data_copy['short_locality_name'] = data_copy['locality_name'].apply(make_short_name)

short_locality_name_keys = data_copy['short_locality_name'].unique()
short_locality_name_keys
data_copy = copy.deepcopy(data)

bad_locality_name = data_copy[data_copy['locality_name'].isna()]

bad_index = bad_locality_name.index
print(bad_index)

data_copy.drop(bad_index, inplace=True)

def make_short_name(row):
    #print(1, row)
    split_name = row.split()
    new_name = []
    for word in split_name:
        if word[0].isupper():
            new_name.append(word)
    new_name = ' '.join(word for word in new_name)
    #print(2, new_name)
    return new_name

data_copy['short_locality_name'] = data_copy['locality_name'].apply(make_short_name)

short_locality_name_keys = data_copy['short_locality_name'].unique()
short_locality_name_keys

**Cоздали список имен для словаря, теперь соотнесем и найдем среднее растояние от названия местности с центром города. Это поможет на заполнить пропуски в locality_name и cityCenters_nearest**

In [25]:
data_copy['short_locality_name'].unique()

In [26]:
locality_and_cityCenter_dict = {}

bad_cityCenters_nearest = data_copy[data_copy['cityCenters_nearest'].isna()]

bad_index = bad_cityCenters_nearest.index
print(bad_index)

data_copy.drop(bad_index, inplace=True)
print(data_copy['short_locality_name'].unique())

def make_locality_and_cityCenter_dict(data_frame):
    for name in short_locality_name_keys:
        mean = data_frame[data_frame['short_locality_name'] == name]['cityCenters_nearest'].mean()
        locality_and_cityCenter_dict[name] = mean
        
make_locality_and_cityCenter_dict(data_copy)
locality_and_cityCenter_dict

**Из проведенной обработки данных видно, что не все объявления имеют расстояния и многие деревни, поселки и города не имеют расстояния от центра Санкт-Петербурга. Для гипотезы по расстоянию от центра важны только квартиры в Санкт-Петербурге. Для этого проверим сколько сейчас пропусков в расстоянии с указанным городом Санкт-Петербург.**

In [27]:
locality_and_cityCenter_dict = {}
short_locality_name_keys = data_copy['short_locality_name'].unique()
make_locality_and_cityCenter_dict(data_copy)
print(locality_and_cityCenter_dict)

SP_data = data.query('locality_name == "Санкт-Петербург" and cityCenters_nearest.isna()')
SP_data['cityCenters_nearest'].isna().value_counts()

**Только для этих 59 значений мы восстановим среднее расстояние от центра города.  
Для объявлений, чье расстояние от центра менее 15 км, заменим пропуск на Санкт-Петебург. Все остальные пропуски зададим расположение "За пределами города".**

In [28]:
data['locality_name'] = data['locality_name'].fillna('empty')
print(data[data['locality_name'] == 'empty']['cityCenters_nearest'].value_counts())

def fill_locality_name(row):
    if row[14] == 'empty':
        if row[16] < 15000:
            row[14] = 'Санкт-Петебург'
        else:
            row[14] = 'За пределами города'
    return row

data = data.apply(fill_locality_name, axis=1)
data.info()

**parks_around3000 и ponds_around3000 заполним значениями 0.**

**Заполнение столцов airports_nearest, cityCenters_nearest и ponds_nearest не требуется из условий гипотез.**

#### Приведение типа данных

Следующие столбцы будут приведены в целочисленный тип(int):

last_price. Обычно никто не ставит цену на квартиру в рублях с указанием копеек.  
floors_total. В домах не может быть половина этажа. Только целые.  
balcony. Исходя из значений в столце, количество балконов должно быть целым.  
airports_nearest, cityCenters_nearest, parks_nearest и ponds_nearest. Здесь расстояния указаны в метрах.  
parks_around3000 и ponds_around3000. Как и в случае с балконами, здесь указывается целое число.  

Значения в airports_nearest, cityCenters_nearest, parks_nearest, ponds_nearest и days_exposition не могут быть изменены на int из-за значений NaN.

In [29]:
data['last_price'] = pd.to_numeric(data['last_price'], downcast='integer')
data['floors_total'] = pd.to_numeric(data['floors_total'], downcast='integer')
data['balcony'] = pd.to_numeric(data['balcony'], downcast='integer')
data['airports_nearest'] = pd.to_numeric(data['airports_nearest'], downcast='integer')
data['cityCenters_nearest'] = pd.to_numeric(data['cityCenters_nearest'], downcast='integer')
data['parks_nearest'] = pd.to_numeric(data['parks_nearest'], downcast='integer')
data['ponds_nearest'] = pd.to_numeric(data['ponds_nearest'], downcast='integer')
data['parks_around3000'] = pd.to_numeric(data['parks_around3000'], downcast='integer')
data['ponds_around3000'] = pd.to_numeric(data['ponds_around3000'], downcast='integer')
data['days_exposition'] = pd.to_numeric(data['days_exposition'], downcast='integer')
data.info()

**Вывод**  
В ходе преодобрабоки данных:

1. Были заполнены значемые строки для гипотез.  
2. Было выполнено удаление ошибчных значений и тех значимых, пустоты которых не возможно было заполнить.  
3. Была произведена смена типов данных.

### Посчитайте и добавьте в таблицу новые столбцы

**Рассчитаем цену одного квадратного метра**

In [30]:
data['price_per_metre'] =(data['last_price'] / data['total_area'] * 100).round()/100
data['price_per_metre']

**Добавим столбцы: день недели, месяц и год публикации.**  
Сначала добавим дополнительный столбец date по first_day_exposition. Просмотрим время последних добавленных объявлений и создадим отдельный столбец времени, который хранит в себе день, месяц и год продажи.


In [31]:
data['date'] =  pd.to_datetime(data['first_day_exposition'], format='%Y-%m-%d')
data['date'].max()


In [32]:
data['weekday'] = data['date']
data['month'] = data['date']
data['year'] = data['date']

def make_weekday_month_year(row):
    row[24] = row[23].weekday()
    row[25] = row[23].month
    row[26] = row[23].year
    return row

data = data.apply(make_weekday_month_year,axis=1)
data.head()

**Добавим тип этажа квартиры: «первый», «последний», «другой»**

In [33]:
def categorize_floor(row):
    if row[8] == 1:
        return 'первый'
    elif row[8] == row[6]:
        return 'последний'
    else:
        return 'другой'
    
data['floor_category'] = data.apply(categorize_floor,axis=1)
data['floor_category'].value_counts() 

In [34]:
data.head()

**Переведем расстояние до центра в "км"**

In [35]:
data['cityCenters_nearest'] = (data['cityCenters_nearest']/1000).round()
data.head()

### Проведите исследовательский анализ данных

Построем гистограммы для следующих параметров:
- общая площадь;
- жилая площадь;
- площадь кухни;
- цена объекта;
- количество комнат;
- высота потолков;
- тип этажа квартиры («первый», «последний», «другой»);
- общее количество этажей в доме;
- расстояние до центра города в метрах;
- расстояние до ближайшего парка

In [36]:
data['total_area'].hist(bins=100)
data['total_area'].describe()

In [37]:
data['total_area'].hist(bins=50, range = (0,150))
data['total_area'].describe()

Больше всех квартир находяться в районе от 30 до 65 кв.м. Выбросы сыше 300 кв.м. обусловлены либо большими планировками либо отдельными частными домами.

In [38]:
data['living_area'].hist(bins=100, range = (0,80))
data['living_area'].describe()

Есть несколько пиков в гистограмме:  
    1. большое кол-во квартир с жилой площадью от 15 до 20 м2  
    2. от 25 до 35 м2  
    3. 45 м2  

In [39]:
data['kitchen_area'].hist(bins=100, range = (0,60))
data['kitchen_area'].describe()

В основном кухни небольшие от 5 до 15 м2

In [40]:
data['price_per_metre'].hist(bins = 100, range = (0, 300000))
data['price_per_metre'].describe()

Общая стоимость в основном в районе 4 миллионов для большинства квартир

In [41]:
data['rooms'].hist(bins=15, range = (0,10))
data['rooms'].describe()

Здесь закономерность имеет верную природы. Работа с данными не требует корректировки.

In [42]:
data['ceiling_height'].hist(bins=10, range = (2,5))
data['ceiling_height'].describe()

In [43]:
data['ceiling_height'].hist(bins=100, range = (10,100))
data['ceiling_height'].describe()

Большинсво кварnир имеют высоту от 2.5 до 2.75 м.  
Избавимся от выбросов по значимым для нас факторам: площадь, цена, число комнат, высота потолков. 
Исключим выбросы согласно значениям на гистограммах

In [45]:
sorted(data[data['ceiling_height'] > 10]['ceiling_height'].unique())

In [46]:
data[data['ceiling_height'] > 10]

In [47]:
data = data.loc[data['ceiling_height'] < 100]

In [48]:
data.loc[data['ceiling_height'] >= 11, 'ceiling_height'] /= 10
sorted(data['ceiling_height'].unique())

In [50]:
data = data.query(
    '12 <= total_area <=  120')
data = data.query(
    '1.118300e+02 <= price_per_metre <=  2e+05')
data = data.query(
    '0 <= rooms <=  6')
data = data.query(
    '2 <= ceiling_height <= 10')


In [51]:
data['floor_category'].hist(bins=30)

In [52]:
data['floors_total'].hist(bins=20, range = (0,30))
data['floors_total'].describe()

In [53]:
data['cityCenters_nearest'].hist(bins=50, range = (0,50))
data['cityCenters_nearest'].describe()

In [54]:
data['parks_nearest'].hist(bins=50, range = (0,1500))
data['parks_nearest'].describe()

**Время продажи квартиры**  
Изучим общие данные по времени продажи квартиры и построем гистограмму.



In [55]:
data['days_exposition'].hist(bins=100)


In [56]:
data['days_exposition'].hist(bins=100, range = (0,500))

In [57]:
data['days_exposition'].hist(bins=45, range=(30,100))

In [58]:
data['days_exposition'].hist(bins=60, range=(100,360))

In [59]:
data['days_exposition'].describe()

Среднее значение - 180 день. Медианное - 96 дней.   
Продаже занимают от 44 до 231 дней  
Быстрые продажи - это продаже менее 7 дней  
Долгие продажи - те которые длятся больше года (360 дней)  

По общей информации на стоимость квартиры влияет ее местоположение (в центра города - на окранине, в хорошем районе - в неблагополучном, близко от местро/транспортных узлов или нет), тип жилья (частный дом - квартира, новый дом - хрущевка/аварийный, наличие лифта или другой инфраструктуры), расположение квартиры в доме (какой этаж, сколько квартир на этаж), площадь и планировка квартиры, вид из окна и общее состояние квартиры (новая, с ремонтом, без ремонта).

Проверим, зависит ли цена квартир от числа комнат, этажа и даты размещения

Составим корреляционные зависимости предложенных факторов.

In [60]:
prise_corr = data.loc[:,['last_price','total_area','living_area','kitchen_area']]
prise_corr.head()

In [61]:
prise_corr.corr()

In [62]:
pd.plotting.scatter_matrix(prise_corr, figsize=(15, 15))

Можно наблюдать зависимость цены от плозащи квартиры, чуть менее эта зависимость наблюдается при сравнении цены с жилой площадью и площадью кухни.  
Большинство квартир продаются в районе от 5 млн. до 10 млн. с площадью от 28 кв.м. до 80 кв.м.

In [63]:
prise_corr_2 = data.loc[:,['last_price', 'rooms', 'cityCenters_nearest']]
prise_corr_2.head()

In [64]:
prise_corr_2.corr()

In [65]:
pd.plotting.scatter_matrix(prise_corr_2, figsize=(15, 15))

Количество также как и площадь не маловажна. Комнаты напрямую определяют размер общей площади. Самые распространенные квартиры имеют от 1 до 3 комнат.  
Дальность от центра уменьшает стоимость достаточно. Это и логично - чем ближе к центру, тем легче до него добраться. Основной поток квартир сконцентрирован на расстоянии до 20 км.

**Изучим зависимость цены от  расположения этажа.**  
Построим гистограммы для каждой категории и выделим укрупненный диапозон с ценой.

In [66]:
data[data['floor_category'] == 'первый']['last_price'].hist(bins = 100, range = (0,17500000))
data[data['floor_category'] == 'первый']['last_price'].describe()

Квартиры на первом этаже не популярны. Больше всего квартир продали от 2.5 млн до 5 млн. Среднее значение 4,4 млн.

In [67]:
data[data['floor_category'] == 'последний']['last_price'].hist(bins = 100, range = (0,17500000))
data[data['floor_category'] == 'последний']['last_price'].describe()

Квартир на последнем этаже также много продали в районе от 2.5 млн до 5 млн. Квартиры на последнем этаже дороже, чем на первом. Среднее значению 5 млн.

In [68]:
data[data['floor_category'] == 'другой']['last_price'].hist(bins = 100, range = (0,17500000))
data[data['floor_category'] == 'другой']['last_price'].describe()

Основные продажи квартир на остальных этажах проходят в районе от 2,5 до 6 млн. Среднее значение ещё выше 5,6 млн. Это говорит о том, что другие этажи определенно стоят дороже, чем первые. Их стоимость в целом выше от квартир на последнем этаже, но нет так сильно. В целом люди платять больше за квартиры не на первом и не на последнем этаже.

**Изучим зависимость от даты размещения.**

In [69]:
prise_weekday_month_year = data.loc[:,['last_price', 'weekday', 'month', 'year']]
prise_weekday_month_year.head()

In [70]:
prise_weekday_month_year.corr()

Матрица корреляции имеет слабые зависимости. Посмотрим на гистограммы по каждому столбцу

In [71]:
prise_weekday_month_year[prise_weekday_month_year['weekday'] == 0]['last_price'].hist(bins=100, range=(0,20000000))
prise_weekday_month_year[prise_weekday_month_year['weekday'] == 1]['last_price'].hist(bins=100, range=(0,20000000), alpha=0.5)


In [72]:
prise_weekday_month_year[prise_weekday_month_year['weekday'] == 2]['last_price'].hist(bins=100, range=(0,20000000))
prise_weekday_month_year[prise_weekday_month_year['weekday'] == 3]['last_price'].hist(bins=100, range=(0,20000000), alpha=0.5)

In [73]:
prise_weekday_month_year[prise_weekday_month_year['weekday'] == 4]['last_price'].hist(bins=100, range=(0,20000000))
prise_weekday_month_year[prise_weekday_month_year['weekday'] == 5]['last_price'].hist(bins=100, range=(0,20000000), alpha=0.5)


In [74]:
prise_weekday_month_year[prise_weekday_month_year['weekday'] == 6]['last_price'].hist(bins=100, range=(0,20000000))


В общей картине стомость квартиры не зависит от дня продажи. Единственное что меняется - это количество проданных квартир. Больше всего квартир продают за будние дни, когда у риелторов и агенств основной режим работы.

Теперь посмотрим гистограммы по месецам

In [75]:
prise_weekday_month_year[prise_weekday_month_year['month'] == 1]['last_price'].hist(bins=100, range=(0,20000000) )
prise_weekday_month_year[prise_weekday_month_year['month'] == 2]['last_price'].hist(bins=100, range=(0,20000000), alpha=0.5)


In [76]:
prise_weekday_month_year[prise_weekday_month_year['month'] == 3]['last_price'].hist(bins=100, range=(0,20000000))
prise_weekday_month_year[prise_weekday_month_year['month'] == 4]['last_price'].hist(bins=100, range=(0,20000000), alpha=0.5)


In [77]:
prise_weekday_month_year[prise_weekday_month_year['month'] == 5]['last_price'].hist(bins=100, range=(0,20000000))
prise_weekday_month_year[prise_weekday_month_year['month'] == 6]['last_price'].hist(bins=100, range=(0,20000000), alpha=0.5)


In [78]:
prise_weekday_month_year[prise_weekday_month_year['month'] == 7]['last_price'].hist(bins=100, range=(0,20000000))
prise_weekday_month_year[prise_weekday_month_year['month'] == 8]['last_price'].hist(bins=100, range=(0,20000000), alpha=0.5)


In [79]:
prise_weekday_month_year[prise_weekday_month_year['month'] == 9]['last_price'].hist(bins=100, range=(0,20000000))
prise_weekday_month_year[prise_weekday_month_year['month'] == 10]['last_price'].hist(bins=100, range=(0,20000000), alpha=0.5)


In [80]:
prise_weekday_month_year[prise_weekday_month_year['month'] == 11]['last_price'].hist(bins=100, range=(0,20000000))
prise_weekday_month_year[prise_weekday_month_year['month'] == 12]['last_price'].hist(bins=100, range=(0,20000000), alpha=0.5)


По графикам видно, что основные продажи так же идут от 2.5 до 7,5 млн каждый месяц. Отличается лишь количество продаж на декабрь и январь. В эти месяцы меньше всего продаж квартир.

Теперь посмотрим по годам

In [81]:
prise_weekday_month_year[prise_weekday_month_year['year'] == 2014]['last_price'].hist(bins=50, range=(0,20000000))
prise_weekday_month_year[prise_weekday_month_year['year'] == 2015]['last_price'].hist(bins=50, range=(0,20000000), alpha=0.5)


In [82]:
prise_weekday_month_year[prise_weekday_month_year['year'] == 2016]['last_price'].hist(bins=50, range=(0,20000000))
prise_weekday_month_year[prise_weekday_month_year['year'] == 2017]['last_price'].hist(bins=50, range=(0,20000000), alpha=0.5)


In [83]:
prise_weekday_month_year[prise_weekday_month_year['year'] == 2018]['last_price'].hist(bins=50, range=(0,20000000))
prise_weekday_month_year[prise_weekday_month_year['year'] == 2019]['last_price'].hist(bins=50, range=(0,20000000), alpha=0.5)


В 2014,2016 и 2019 году кол-во продаж гораздо меньше чем в другие годы

Также имеется небольшое смещение пика вправо. Легкая положительная зависимость есть, она исходит из инфляции в стране.

**Выберите 10 населённых пунктов с наибольшим числом объявлений.**
1. Посчитайте среднюю цену квадратного метра в этих населённых пунктах.
2. Выделите населённые пункты с самой высокой и низкой стоимостью жилья 

Для оптимизации поиска названия применим функцию для создания коротких имен, чтобы исключить дубликаты.

In [84]:
data['short_locality_name'] = data['locality_name'].apply(make_short_name)
top_10_locality_name = data['short_locality_name'].value_counts()[:10]
print(top_10_locality_name)
top_10_locality_name = top_10_locality_name.index
top_10_locality_name

In [85]:
top_10_pivot_table_mean = (data
                      .query('short_locality_name in @top_10_locality_name')
                      .pivot_table(index='short_locality_name', values='price_per_metre', aggfunc='mean')
                      .sort_values('price_per_metre', ascending=False))
top_10_pivot_table_mean

In [86]:
top_10_pivot_table_min_max = (data
                      .query('short_locality_name in @top_10_locality_name')
                      .pivot_table(index='short_locality_name', values='price_per_metre', aggfunc=['min','max']))
top_10_pivot_table_min_max

**Задание "Ранее вы посчитали расстояние до центра в километрах.**
1. Вычислите среднюю цену каждого километра
2. Постройте график изменения средней цены для каждого километра от центра Петербурга.

In [87]:
SP_data = data.query('locality_name == "Санкт-Петербург"')
SP_data.info()

In [88]:
price_per_km = (SP_data
               .pivot_table(index='cityCenters_nearest', values='price_per_metre', aggfunc='mean'))
price_per_km

In [89]:
plot_price_per_km = price_per_km
plot_price_per_km['km'] = price_per_km.index
plot_price_per_km

In [90]:
price_per_km.plot(x='km', y='price_per_metre' ,kind='line')

Центральной зоной можно считать квартиры в радиусе 3 км от цента Санкт-Петербурга. Локальный пик в районе 5-7 км обусловлен инфраструктурой. Локальный пик в 27 км обусловлен дороговизной квартир в Петергофе.

**Вывод**

Проведя анализ данных по зависимостям цены за квадратный метр по отношению к другим факторам имеются следующие выводы:

- Для проживания в городе люди пытаются найти квартиры стоимостью от 2,5 до 5,5 млн. с как минимум одной комнатой, потолком в 2.7 метра и планировкой от 40 кв.м.
- -Основные продажи квартир случаются через 30 - 200 дней. Большнство обусловлены планами риэлторов и системами продаж.
- На стоимость квартиры положительно влияет общая площадь, количество комнат, год продажи, отрицательно - удаленность от центра, день и месяц продажи.
- Санкт-Петербур имеет самую большую стоимость и самую большую стоимость за кв.м.
- В центре квартиры в целом лучше и их цена оправдана их местоположением.

### Общий вывод

- Были определены аномалии и построена стратегия замены пропусков.
- Пропуски были заполнены максимально возможными способами, чтобы каждое значение могло влиять на гипотезы.
- Приведены типы данных для уменьшения используемой памяти и отображаемого вида в таблице.
- Созданы дополнительные столбцы для улучшения анализа данных.
- Даны ответы на заданные вопросы.

Основные параметры для оценки стоимости недвижимости:

1. Основным ориентиром для определения стоимости квартиры - это стоимость квадратного метра.
2. Общая площадь, отношения жилой к общей и отношение кухонной к общей площади помогут определить самый предпочитаемый тип квартир.
3. Даты продаж позволят определить, в какой момент лучше продвигать рекламную компанию по продаже домов. В какие месяцы или дни недели. Количество продаж по годам имеет естественную зависимость от количество жителей города.
4. Этаж квартиры. В большинстве случаев люди редко берут квартиры на первм этаже из-за их небезопасности, а квартиры на последним - из-за возможных проблем с крышей. Соотвественно самыми дорогими считаются комфортные квартиры между первым и последним этажом.
5. Пригороды к Санкт-Петербургу имеют тенденцию к снижению цен за квартиру ввиду отсуствия престижа.
6. Центр города в целом имеет хорошие квартиры и планировки и соотвественно одни из самых больших по стоимости квартир в районе.